In [6]:
# python 3.8 required!
%pip install scikit-learn
%pip install cython
%pip install pandas
%pip install numpy
%pip install rankfm

from rankfm.rankfm import RankFM
from rankfm.evaluation import (discounted_cumulative_gain, diversity, hit_rate,
                               precision, recall, reciprocal_rank)
from pandas import DataFrame
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
"""
global vars
"""
rating_converter = {
    'user': int,
    'isbn': str,
    'rating': float
}

reader_config = {
    'encoding': 'utf-8',
    'separator': ';',
}

ratings = pd.read_csv('../data/BX-Book-Ratings-cleaned.csv', sep=reader_config['separator'], converters=rating_converter, encoding=reader_config['encoding'])
mean_rating = ratings.loc[ratings['rating'] > 0]['rating'].mean()
mean_rating_per_user = ratings.loc[ratings['rating'] > 0].select_dtypes(include=['int64', 'float64']).groupby('user').mean()
user_features = pickle.load(open("user_features", "rb"))
book_features = pickle.load(open("book_features", "rb"))

def good_ratings_and_implicit_filter(group):
    avg_rating = group['rating'].mean()
    
    return group.loc[(group['rating'] >= avg_rating) | (group['rating'].eq(0))]


In [11]:
import math
import os
import pickle

min_10_r_per_u = ratings.groupby('user').filter(lambda r: r['rating'].count() >= 10)[["user", "isbn", "rating"]]
min_10_r_per_u = min_10_r_per_u.reset_index()

model = RankFM(factors=50, loss='warp', max_samples=100, alpha=0.01, sigma=0.1, learning_rate=0.10, learning_schedule='invscaling')
stratified_kfold = StratifiedKFold(n_splits=2)

all_precisions = []
all_recalls = []


def rank_at_k(model: RankFM, test_set: DataFrame, k=10):
    """
    Calculates mean recall at k recommendations.

    -- Important Definitions --
    False Negatives: Not Recommended but should have, bc has good rating or implicit fb
    """
    test_set = test_set.groupby('user').apply(good_ratings_and_implicit_filter).reset_index(drop=True)
    interactions_per_user = test_set.groupby('user')['isbn'].apply(set)
    test_users = test_set['user'].unique()
    recs: DataFrame = model.recommend(users=test_users, n_items=k,filter_previous=True, cold_start='nan')
    user_in_recs = recs.index.values

    totalndcg = []
    for user in user_in_recs:
        recommended_books = set(recs.loc[user])
        gt_books = interactions_per_user[user]

        dcg = 0
        idcg = 0
        tpcount = 0

        for ind, b in enumerate(recommended_books):
            for r in gt_books:
                if r == b:
                    tpcount += 1
                    if(ind >= 2):
                        dcg += 1/math.log2(ind)
        for ind in range(1, tpcount):
            idcg += 1/math.log2(ind+1)
        
        if(tpcount>0):
            idcg += 1
            totalndcg.append(dcg/idcg)
    
    return np.mean(totalndcg)

cont = True

print(f"Fold {i}:")
train_set = min_10_r_per_u.iloc[train_index]
test_set = min_10_r_per_u.iloc[test_index]

train_user_features = user_features[user_features["User-ID"].isin(train_set['user'])]
train_user_features.loc[:,"Age"] /= train_user_features["Age"].max()
train_book_features = book_features[book_features["ISBN"].isin(train_set['isbn'])]
train_book_features.loc[:,"bf_1"] /= train_book_features['bf_1'].max()

weights =[]

for i, row in train_set.iterrows():
    if row.rating == 0:
        weights.append(1)
    elif row.rating >= mean_rating_per_user.loc[row.user].item():
        weights.append(1)
    else:
        weights.append(0)

print("fitting...")
model = pickle.load(open("../backend/app/recommender/lab2_all_ratings_with_binary_weights"))

print("validating...")
print("ncdg:",rank_at_k(model=model, test_set=min_10_r_per_u, k=10))

/home/stefan/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


Fold 0:


/tmp/ipykernel_10891/1456118734.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_user_features.loc[:,"Age"] /= train_user_features["Age"].max()
/tmp/ipykernel_10891/1456118734.py:61: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  train_book_features.loc[:,"bf_1"] /= train_book_features['bf_1'].max()
/tmp/ipykernel_10891/1456118734.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_book_

fitting...
validating...


ValueError: too many values to unpack (expected 2)